In [1]:
import os 
import pandas as pd 
import numpy as np
from boruta import BorutaPy
from preprocessing import imputation_scaling

from sklearn.ensemble import RandomForestClassifier

# Boruta

In [3]:
datasetTimepoint = "12_uponAdmission"
target = "disease_severity"

''' 
Define paths
'''
dataPath = "../../results/preprocessing/cleaned"
dataset = f"{datasetTimepoint}_{target}_cleaned.csv"
resultsPath = f"../../results/featureSelection/{datasetTimepoint}"
os.makedirs(resultsPath, exist_ok=True)

In [5]:
''' 
Read data
'''
data = pd.read_csv(f"{dataPath}/{dataset}", index_col=0)
X = data.drop(target, axis=1)
y = data[target].ravel()

'''
Prepare preprocessing
'''
num_columns = X.select_dtypes(include=["float64"]).columns
bin_columns = X.select_dtypes(include=["int64"]).columns
cat_columns = X.select_dtypes(include=["object"]).columns
preprocessor = imputation_scaling(num_columns, bin_columns, cat_columns, X)
columnOrderAfterPreprocessing = [ele[5:] for ele in preprocessor.get_feature_names_out()]

''' 
Preprocess
'''
X_preproc = preprocessor.fit_transform(X)

print(X_preproc.shape)

(164, 77)


In [16]:
''' 
Boruta
'''

perc = 100

rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=None, perc=perc)
# find all relevant features
feat_selector.fit(X_preproc, y)


''' 
Get selected variables
'''
np.array(columnOrderAfterPreprocessing)[feat_selector.support_]

array(['the_main_reason_for_hospital_admission',
       'saturation_measured_based_on…', 'ferritin_admission_µg/L',
       'ldh_admission_µkat/L', 'xray_admission_pathological_n/y',
       'xray_admission_thickenings', 'dexamethasone',
       'dexamethasone_duration_th_days', 'crp___max', 'leukocyte_max',
       'neutrophils', 'ferritin_max', 'ldh_max', 'maximum_oxygen_demand',
       'treatment_attempt_for_covid19_n/y'], dtype='<U45')

In [17]:
print(len(np.array(columnOrderAfterPreprocessing)[feat_selector.support_]))
[ele for ele in np.array(columnOrderAfterPreprocessing)[feat_selector.support_]]

15


['the_main_reason_for_hospital_admission',
 'saturation_measured_based_on…',
 'ferritin_admission_µg/L',
 'ldh_admission_µkat/L',
 'xray_admission_pathological_n/y',
 'xray_admission_thickenings',
 'dexamethasone',
 'dexamethasone_duration_th_days',
 'crp___max',
 'leukocyte_max',
 'neutrophils',
 'ferritin_max',
 'ldh_max',
 'maximum_oxygen_demand',
 'treatment_attempt_for_covid19_n/y']

# Save outcome

In [18]:
with open(f"{resultsPath}/boruta.txt", "w") as f: 
    for ele in np.array(columnOrderAfterPreprocessing)[feat_selector.support_]:
        f.write(ele+"\n")